# 1. Peridically fetch traffic trends and attack spikes from clodfare and/or IPS from abuse IPDP
# 2. Classify IPS with high DDos confidence score using machine learning
# 3, Remote IPs to coordinate and show them in fromtend globe

In [1]:
import pandas as pd
import numpy as np
import requests

In [2]:
import requests 

API_TOKEN = "kvVDk6bPlZ8WE305dH5EqtcMTG8tf0RkCvzJIeqL"
url="https://api.cloudflare.com/client/v4/user/tokens/verify"

headers={
    "Authorization": f"Bearer {API_TOKEN}"
}

response=requests.get(url=url,headers=headers)
# print(response.status_code)
# print(response.json())

In [12]:
API_TOKEN = "kvVDk6bPlZ8WE305dH5EqtcMTG8tf0RkCvzJIeqL"
HEADERS = {"Authorization": f"Bearer {API_TOKEN}"}

PARAMS = {
    "aggInterval": "1h",
    "dateRange": "1d",
    "format": "json"
}

def fetch_data(url):
    response = requests.get(url, headers=HEADERS, params=PARAMS)
    if response.status_code == 200:
        return response.json().get("result", {})
    else:
        print(f"Error {response.status_code}: {response.text}")
        return []

# Layer 3 URL
vector_url = "https://api.cloudflare.com/client/v4/radar/attacks/layer3/timeseries_groups/vector"
ip_url="https://api.cloudflare.com/client/v4/radar/attacks/layer3/timeseries_groups/ip_version"
protocol_url="https://api.cloudflare.com/client/v4/radar/attacks/layer3/timeseries_groups/protocol"
bitrate_url="https://api.cloudflare.com/client/v4/radar/attacks/layer3/timeseries_groups/bitrate"
duration_url="https://api.cloudflare.com/client/v4/radar/attacks/layer3/timeseries_groups/duration"
# Fetch L3 Data
vector_data = fetch_data(vector_url)
ip_data=fetch_data(ip_url)
protocol_data=fetch_data(protocol_url)
bitrate_data=fetch_data(bitrate_url)
duration_data=fetch_data(duration_url)


# Convert to DataFrame
df_vector = pd.DataFrame(vector_data)
df_ip = pd.DataFrame(ip_data)
df_protocol = pd.DataFrame(protocol_data)
df_bitrate = pd.DataFrame(bitrate_data)
df_duration = pd.DataFrame(duration_data)
# print(df_ip)


In [16]:
df_vector.head(1)

,serie_0,meta
timestamps,"[2025-08-14T10:00:00Z, 2025-08-14T11:00:00Z, 2...",NaN


In [15]:
df_ip.head(1)

,serie_0,meta
timestamps,"[2025-08-14T10:00:00Z, 2025-08-14T11:00:00Z, 2...",NaN


In [5]:
import pandas as pd

# 'df_vector' is the DataFrame you just printed
timestamps = df_vector.loc['timestamps'].values[0]  # Extract timestamps

# Convert to long format
records = []
for attack_type in df_vector.index:
    if attack_type in ["timestamps", "dateRange", "confidenceInfo", "aggInterval", "normalization", "lastUpdated", "units", "meta"]:
        continue
    values = df_vector.loc[attack_type].values[0]
    for t, v in zip(timestamps, values):
        records.append({"datetime": t, "attack_type": attack_type, "value": v})

df_long = pd.DataFrame(records)
print(df_long)


                 datetime        attack_type      value
0    2025-08-14T03:00:00Z  Mirai (UDP) Flood   46.45641
1    2025-08-14T04:00:00Z  Mirai (UDP) Flood  23.368166
2    2025-08-14T05:00:00Z  Mirai (UDP) Flood  57.965298
3    2025-08-14T06:00:00Z  Mirai (UDP) Flood  54.319646
4    2025-08-14T07:00:00Z  Mirai (UDP) Flood  35.794531
..                    ...                ...        ...
235  2025-08-14T22:00:00Z         ICMP Flood   0.014012
236  2025-08-14T23:00:00Z         ICMP Flood   0.005791
237  2025-08-15T00:00:00Z         ICMP Flood   0.005936
238  2025-08-15T01:00:00Z         ICMP Flood   0.017949
239  2025-08-15T02:00:00Z         ICMP Flood   0.043204

[240 rows x 3 columns]


In [6]:
df_long["attack_type"].value_counts()

attack_type
Mirai (UDP) Flood    24
UDP Flood            24
SYN Flood            24
ACK Flood            24
DNS Amplification    24
other                24
TCP Flood            24
DNS Flood            24
NTP Amplification    24
ICMP Flood           24
Name: count, dtype: int64

In [9]:
df_long[df_long["attack_type"]=="Mirai (UDP) Flood"]

,datetime,attack_type,value
0,2025-08-14T03:00:00Z,Mirai (UDP) Flood,46.45641
1,2025-08-14T04:00:00Z,Mirai (UDP) Flood,23.368166
2,2025-08-14T05:00:00Z,Mirai (UDP) Flood,57.965298
3,2025-08-14T06:00:00Z,Mirai (UDP) Flood,54.319646
4,2025-08-14T07:00:00Z,Mirai (UDP) Flood,35.794531
5,2025-08-14T08:00:00Z,Mirai (UDP) Flood,11.033363
6,2025-08-14T09:00:00Z,Mirai (UDP) Flood,5.106559
7,2025-08-14T10:00:00Z,Mirai (UDP) Flood,6.99333
8,2025-08-14T11:00:00Z,Mirai (UDP) Flood,20.12028
9,2025-08-14T12:00:00Z,Mirai (UDP) Flood,55.904404
